In [114]:
from transformers import BertTokenizer
from razdel import sentenize
import torch
import numpy as np
import pandas as pd
import tqdm
import json
import pickle
import os
import io

In [3]:
DEVICE = 'cuda:4'

## Dataset

In [64]:
from torch.utils.data import Dataset, DataLoader

In [131]:
class TelegramRegressionReader(Dataset):
    def __init__(self, txt_path, vec_path, chunk_size=2048):
        self.txt_path = txt_path
        self.vec_path = vec_path

        self.shift = 3200 # numpy load reads 3200 bytes from file handler which is equal one vector
        
        self.chunk_size = chunk_size
        
        self.size = sum(
            len(el) for el in pd.read_json(
                self.txt_path,
                encoding='utf-8',
                lines=True,
                chunksize=chunk_size)
        )
        
        s = [0]
        with open(txt_path, 'r', encoding='utf-8') as f:
            self.txt_linelocs = [s.append(s[0]+len(n)) or s.pop(0) for n in f]

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        assert type(idx) == int

        with open(self.txt_path, 'r', encoding='utf-8') as f_txt,\
             open(self.vec_path, 'rb') as f_vec:
            f_txt.seek(self.txt_linelocs[idx], 0)
            txt = f_txt.readline()
            
            f_vec.seek(self.shift * idx, 0)
            vec = np.load(f_vec).reshape(1, -1)
            
            sample = {'text': txt, 'vector': vec}

        return sample

In [132]:
data = TelegramRegressionReader('/data/alolbuhtijarov/datasets/BertSumAbs_predictions/texts.jsonl',
                                '/data/alolbuhtijarov/datasets/BertSumAbs_predictions/vectors.npy')

In [143]:
%%time
sample = data[111052]

CPU times: user 3.14 ms, sys: 522 µs, total: 3.67 ms
Wall time: 2.36 ms


In [144]:
sample['vector'].mean(), sample['vector'].std(), sample['vector'].max(), sample['vector'][0][::100] 

(0.0047325664,
 0.3477171,
 1.1353046,
 array([-0.06912806, -0.3473212 ,  0.01208007,  0.21316218, -0.11906805,
         0.11900068, -0.16061948, -0.18539721], dtype=float32))

In [145]:
json.loads(sample['text'])['text'] + ' ' + json.loads(sample['text'])['title']

'тегеран. на следующей неделе национальная иранская нефтяная компания (nioc) планирует выставить на продажу 6 млн баррелей нефти и газового конденсата. аукцион будет проходить на иранской энергетической бирже (irenex). в понедельник nioc предложит 2 млн баррелей газового конденсата по базовой цене $57,87 за баррель. на следующий день, 19 ноября, компания продаст 2 млн баррелей легкой нефти по базовой цене $ 56,72 за баррель, а в среду, 20 ноября, выставит на продажу 2 млн баррелей тяжелой нефти, уточняет «иран.ру». закон о бюджете ирана на текущий 1398 календарный год, который начался 21 марта, обязывает министерство нефти ирана продавать на irenex не менее 2 млн баррелей легкой сырой нефти, 2 миллиона баррелей тяжелой сырой нефти и 2 млн баррелей газового конденсата. nioc планирует выставить на продажу 6 млн баррелей нефти и газового конденсата'

### Encoder with pretrained FastText embeddings

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        vec = list(map(float, tokens[1:]))
        vec = np.array(vec)
        data[tokens[0]] = vec
    return data

In [ ]:
ft_vecs = load_vectors("/data/alolbuhtijarov/wiki-news-300d-1M-subword.vec")

In [ ]:
vocab_token_vectors = torch.FloatTensor([
    ft_vecs.get(w) if w in ft_vecs else np.random.rand(300) * 1e-3 for w in tokens
])

vocab_token_vectors.shape

In [ ]:
class SmallEncoder(nn.Module):
    def __init__(self, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=128,
                freeze_embed=True):
        super().__init__()
        
        self.embed = nn.Embedding.from_pretrained(vocab_token_vectors, freeze=freeze_embed)

        self.layer_title = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU(),
        )

        self.layer_text = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU()
        )

        self.layer_cat = nn.Sequential(
            nn.Linear(n_cat_features, hid_size * 2),
            nn.BatchNorm1d(num_features=hid_size * 2),
            nn.ReLU(),
            nn.Linear(hid_size * 2, hid_size),
            nn.BatchNorm1d(num_features=hid_size),
            nn.ReLU()
        )

        self.predictor = nn.Linear(300 * 2 + hid_size, 1)


    def forward(self, batch):
        title = self.embed(batch["Title"])
        title = title.permute(0, 2, 1)
        title = self.layer_title(title).squeeze(-1)

        text = self.embed(batch["FullDescription"])
        text = text.permute(0, 2, 1)
        text = self.layer_text(text).squeeze(-1)

        cat = self.layer_cat(batch["Categorical"])

        x = torch.cat([title, text, cat], dim=1)
        return self.predictor(x).squeeze(-1)

https://pytorch.org/docs/stable/optim.html#per-parameter-options

In [ ]:
def separate_optimizer(net):
    embed_param = [kv[1] for kv in net.named_parameters() if kv[0] == 'embed.weight']
    model_params = [kv[1] for kv in net.named_parameters() if kv[0] != 'embed.weight']
    opt = torch.optim.Adam([
                {'params': model_params},
                {'params': embed_param, 'lr': 3e-4}
    ], lr=3e-3)
    return opt